In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"theksg9","key":"5783530dd4c30706555b3c3f51310057"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

100% 2.70G/2.70G [00:41<00:00, 25.0MB/s]
100% 2.70G/2.70G [00:41<00:00, 70.3MB/s]


In [ ]:
from zipfile import ZipFile
file_name="/content/new-plant-diseases-dataset.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()

In [ ]:
!mkdir dataset/potato dataset/potato/train dataset/potato/test dataset/potato/train/p_Early_Blight dataset/potato/train/p_Late_Blight dataset/potato/train/p_Healthy dataset/potato/test/p_Early_Blight dataset/potato/test/p_Late_Blight dataset/potato/test/p_Healthy

In [ ]:

import cv2
def controller(img, brightness=255, contrast=127):
    brightness = int((brightness - 0) * (255 - (-255)) / (510 - 0) + (-255))

    contrast = int((contrast - 0) * (127 - (-127)) / (254 - 0) + (-127))

    if brightness != 0:

        if brightness > 0:

            shadow = brightness

            max = 255

        else:

            shadow = 0
            max = 255 + brightness

        al_pha = (max - shadow) / 255
        ga_mma = shadow

        # The function addWeighted
        # calculates the weighted sum
        # of two arrays
        cal = cv2.addWeighted(img, al_pha,
                              img, 0, ga_mma)

    else:
        cal = img

    if contrast != 0:
        Alpha = float(131 * (contrast + 127)) / (127 * (131 - contrast))
        Gamma = 127 * (1 - Alpha)

        # The function addWeighted calculates
        # the weighted sum of two arrays
        cal = cv2.addWeighted(cal, Alpha,
                              cal, 0, Gamma)

    # putText renders the specified
    # text string in the image.


    return cal

In [ ]:
import glob
files=glob.glob("/content/dataset/potato/test/p_Late_Blight/*.*")

In [ ]:
len(files)

485

In [ ]:
c=0
for f in files:
  img=cv2.imread(f)
  g=controller(img,299,146)
  path="/content/dataset/potato/train/p_Healthy/"+str(c)+".jpg"
  cv2.imwrite(path,g)
  c+=1

In [ ]:
!mv /content/dataset/dataset/train/Potato___Early_blight/*.* /content/dataset/potato/train/p_Early_Blight
!mv /content/dataset/dataset/valid/Potato___Early_blight/*.* /content/dataset/potato/test/p_Early_Blight

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale = 1./255,
	      rotation_range=90,
        brightness_range=[0.6,1.0],zoom_range=[0.8,1],
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
        )

train_set=train_datagen.flow_from_directory(
    '/content/dataset/potato/train',
    target_size=(256,256),
    batch_size=64,
    class_mode='categorical'
)

test_datagen=ImageDataGenerator(rescale=1./255)

test_set=test_datagen.flow_from_directory(
    '/content/dataset/potato/test',
    batch_size=16,
    target_size=(256,256),
    class_mode='categorical'
)

Found 5702 images belonging to 3 classes.
Found 1426 images belonging to 3 classes.


In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
classifier=Sequential()
classifier.add(Convolution2D(16,(3,3),activation='relu',input_shape=(256,256,3)))
classifier.add(MaxPooling2D())
classifier.add(Convolution2D(32,(3,3),activation='relu',input_shape=(256,256,3)))
classifier.add(MaxPooling2D())
classifier.add(Flatten())
classifier.add(Dense(512,activation='relu'))
classifier.add(Dense(0.3))
classifier.add(Dense(128,activation='relu'))
classifier.add(Dense(0.3))
classifier.add(Dense(3,activation='softmax'))

classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
classifications=classifier.fit_generator(
    train_set,
    steps_per_epoch=5702/64,
    epochs=20,
    validation_data=test_set,
    validation_steps=1426/16,
    
    verbose=2
)